In [1]:
 %load_ext autotime

In [2]:
# import sys
# sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')
# sys.path.append('/scratch/pradap/python-work/enrique')

time: 994 µs


In [3]:
import magellan as mg
import pandas as pd

DEBUG:Cloud:Log file (/Users/pradap/.picloud/cloud.log) opened


time: 3.14 s


In [4]:
# initialize JVM
mg.init_jvm()

True

time: 140 ms


In [5]:
# Read walmart books data
wal =  mg.read_csv(mg.get_install_path()+'/datasets/books/walmart.csv',
                    dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                    low_memory=False, key='id')

time: 1.7 s


In [6]:
# Read bowker books data
bwk = mg.read_csv(mg.get_install_path()+'/datasets/books/bowker.csv', 
                  dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                  low_memory=False, key='id')


time: 155 ms


In [7]:
# Create Attribute equivalence blocker and block using isbn
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(wal, bwk, 'isbn', 'isbn', ['title', 'author'], ['title', 'author'])


time: 299 ms


In [8]:
# Write candidate set to disk -- end of project stage 2
C.to_csv('books_candset.csv')

True

time: 3.87 ms


In [9]:
# Read candidate set 
C = mg.read_csv('books_candset.csv', ltable=wal, rtable=bwk)

time: 4.36 ms


In [10]:
# print number of tuples in candidate set
len(C)

68

time: 2.04 ms


In [11]:
#plan

# 1. sample candidate set --> S
# 2. label S --> L
# 3. Two matching options
#    a. ML matcher + Trigger
#    b. Rule-based matcher + Trigger
# 4. Choose one of the above two options (which ever produces better result), and 
#     apply it over candidate set

time: 1.96 ms


In [12]:
# sample table
S = mg.sample_table(C, 50)

time: 2.74 ms


In [13]:
# label table
# L = mg.label_table(S, 'gold')

time: 1.05 ms


In [14]:
# for demo reading a pre-labeled file
L = mg.read_csv('label_table_demo_books.csv', ltable=wal, rtable=bwk)

time: 4.98 ms


In [15]:
# ML matcher + Trigger
#   1. Find best possible ML matcher
#       1. Get features for matching
#       2. Extract feature vectors
#       3. Select matcher using cross validation
#       4. Debug matcher
#       5. Get the best possible matcher
#   2. Add Triggers 

time: 3.65 ms


In [16]:
# get features for matching
feature_table = mg.get_features_for_matching(wal, bwk)

time: 3.47 s


In [17]:
#feature_table

time: 1.54 ms


In [18]:
# select a subset of matchers
f = feature_table.ix[[3,7,18,26, 53]]

time: 1.73 ms


In [19]:
# display selected features
f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
3,title_title_jac_qgm_3_qgm_3,title,title,qgm_3,qgm_3,jaccard,<function title_title_jac_qgm_3_qgm_3 at 0x120...,from magellan.feature.simfunctions import *\nf...
7,author_author_jac_qgm_3_qgm_3,author,author,qgm_3,qgm_3,jaccard,<function author_author_jac_qgm_3_qgm_3 at 0x1...,from magellan.feature.simfunctions import *\nf...
18,binding_binding_jac_qgm_3_qgm_3,binding,binding,qgm_3,qgm_3,jaccard,<function binding_binding_jac_qgm_3_qgm_3 at 0...,from magellan.feature.simfunctions import *\nf...
26,publisher_publisher_jac_qgm_3_qgm_3,publisher,publisher,qgm_3,qgm_3,jaccard,<function publisher_publisher_jac_qgm_3_qgm_3 ...,from magellan.feature.simfunctions import *\nf...
53,lang_lang_lev,lang,lang,None,None,lev,<function lang_lang_lev at 0x11fe442a8>,from magellan.feature.simfunctions import *\nf...


time: 18.9 ms


In [20]:
# Extract feature vectors
G = mg.extract_feat_vecs(L, feature_table=f, attrs_after='gold')

time: 471 ms


In [21]:
G.head(10)

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold
0,0,4086892,9780226156439,0.785714,1.000000,0,0.518519,1,1
1,1,535599,9780060655808,0.222222,1.000000,0,0.320000,1,1
2,2,1386538,9780130901262,1.000000,0.785714,1,0.733333,1,1
3,3,15280600,9780217455008,0.720000,0.666667,1,1.000000,1,1
4,4,15286798,9780217825382,1.000000,1.000000,1,1.000000,1,1
5,5,15295588,9780217940207,0.308271,0.695652,1,0.083333,1,0
6,6,15334872,9780217270052,1.000000,1.000000,1,1.000000,1,1
7,7,15338150,9780217304399,0.280702,1.000000,1,1.000000,1,1
8,8,1566948,9780070479081,0.822222,0.260870,1,0.303030,1,1
9,9,4564015,9780080446837,1.000000,NaN,1,0.512821,1,0


time: 40.1 ms


In [22]:
# G contains NaN. 
# Impute values
G.fillna(0, inplace=True)

time: 1.92 ms


In [23]:
G.head(10)

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold
0,0,4086892,9780226156439,0.785714,1.000000,0,0.518519,1,1
1,1,535599,9780060655808,0.222222,1.000000,0,0.320000,1,1
2,2,1386538,9780130901262,1.000000,0.785714,1,0.733333,1,1
3,3,15280600,9780217455008,0.720000,0.666667,1,1.000000,1,1
4,4,15286798,9780217825382,1.000000,1.000000,1,1.000000,1,1
5,5,15295588,9780217940207,0.308271,0.695652,1,0.083333,1,0
6,6,15334872,9780217270052,1.000000,1.000000,1,1.000000,1,1
7,7,15338150,9780217304399,0.280702,1.000000,1,1.000000,1,1
8,8,1566948,9780070479081,0.822222,0.260870,1,0.303030,1,1
9,9,4564015,9780080446837,1.000000,0.000000,1,0.512821,1,0


time: 49.9 ms


In [24]:
# Create ML matchers
dt = mg.DTMatcher(name='DecisionTree', random_state=0)
svm = mg.SVMMatcher()
rf = mg.RFMatcher(name='RF', random_state=0)
nb = mg.NBMatcher(name='NB')
lg = mg.LogRegMatcher(name='LogReg')
ln = mg.LinRegMatcher(name='LinReg')

time: 3.86 ms


In [25]:
dt.name

'DecisionTree'

time: 5.51 ms


In [26]:
svm.name

'SVM_63381995088636346441'

time: 7.59 ms


In [27]:
# select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg], table=G, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision') 
        # metric can take precision, recall, f1

time: 72.8 ms


In [28]:
# Check the selected matcher
result['selected_matcher']

time: 2.03 ms


In [29]:
# Display cross validation statistics
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object a...,5,0.875000,0.857143,0.875000,0.700000,1.000000,0.861429
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,1.000000,0.900000,0.888889,0.750000,0.714286,0.850635
2,SVM_63381995088636346441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.777778,1.000000,0.666667,0.700000,0.700000,0.768889
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,0.700000,0.555556,0.900000,0.777778,0.800000,0.746667
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.800000,0.875000,0.800000,0.666667,1.000000,0.828333


time: 56.5 ms


In [30]:
# debug: understand why DT matcher is not performing well

# Plan : 
# Split data into 'train' and 'test'. 
# Train matcher using 'train' set
# Inspect matcher using 'test' set

time: 8.26 ms


In [31]:
# Split feature vectors to train and test
train_test = mg.train_test_split(G, train_proportion=0.7, random_state=0)
train = train_test['train']
test = train_test['test']

time: 8.24 ms


In [32]:
# mg.vis_debug_dt(dt, train, test, 
#         exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
#         target_attr='gold') 

time: 2.71 ms


In [33]:
# Experimental feature - requires installation of graphviz : www.graphviz.org
#mg.visualize_tree(dt, G.columns, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'])

time: 924 µs


In [34]:
#from IPython.display import Image
#Image(filename='dt_.png') 

time: 2.06 ms


In [35]:
# mg.vis_debug_rf(rf, train, test, 
#         exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
#         target_attr='gold') 

time: 2.7 ms


In [36]:
# Try adding all the features
H = mg.extract_feat_vecs(L, feature_table=feature_table, attrs_after='gold')

time: 1e+03 ms


In [37]:
# select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg], table=G, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision') 

time: 94.6 ms


In [38]:
result['selected_matcher']

time: 4.27 ms


In [39]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object a...,5,0.714286,1.000000,1.000,0.666667,0.875000,0.851190
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0.833333,0.714286,0.875,0.777778,1.000000,0.840079
2,SVM_63381995088636346441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.700000,0.900000,0.600,1.000000,0.714286,0.782857
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,0.555556,0.800000,0.800,0.700000,0.888889,0.748889
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.777778,0.875000,0.875,0.700000,0.800000,0.805556


time: 21.1 ms


In [40]:
# Choose DT matcher (with subset of features)

# Add triggers to DT Matcher
# Ideal plan
# Split G
# G :------------------|----------------------------|--------------------|
#   [development(60%): train(30%)  + validation(30%)]     [test(40%)]
# Use train + validation to come up with triggers
# Use test set to evaluate ML matcher + triggers


time: 2.5 ms


In [41]:
# Split G into development and test
development_test = mg.train_test_split(G, train_proportion=0.6, 
                                       random_state=0)
development = development_test['train']
test = development_test['test']

time: 4.41 ms


In [42]:
len(development), len(test)

(30, 20)

time: 2.21 ms


In [43]:
development.head(2)

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold
15,15,5087512,9780028348261,0.534483,0.6,1,0.264706,1,1
30,30,14022273,9780217263016,0.408759,1.0,1,1.000000,1,1


time: 13 ms


In [44]:
# First check, how does DT performs with out any triggers
# Train using development
dt.fit(table=development, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')

time: 7.44 ms


In [45]:
# Test using test
I = dt.predict(table=test, 
              exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
              target_attr='predicted',
              append=True, inplace=False)

time: 13.5 ms


In [46]:
# evaluate the result.
eval_summary = mg.eval_matches(I, 'gold', 'predicted')

time: 10.9 ms


In [78]:
len(I)

20

time: 1.82 ms


In [47]:
eval_summary.keys()

['prec_numerator',
 'prec_denominator',
 'precision',
 'recall_numerator',
 'recall_denominator',
 'recall',
 'f1',
 'pred_pos_num',
 'false_pos_num',
 'false_pos_ls',
 'pred_neg_num',
 'false_neg_num',
 'false_neg_ls']

time: 2.51 ms


In [48]:
mg.print_eval_summary(eval_summary)

Precision : 93.33% (14/15)
Recall : 82.35% (14/17)
F1 : 87.5%
False positives : 1 (out of 15 positive predictions)
False negatives : 3 (out of 5 negative predictions)
time: 1.29 ms


In [49]:
# Split development into train and validation set. Use them to debug.

train_validation = mg.train_test_split(development, train_proportion=0.5, random_state=10)
# mg.vis_debug_dt(dt, train_validation['train'], train_validation['test'], 
#                exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
#                target_attr='gold')


time: 4.74 ms


In [50]:
# Add triggers to address false negatives
# if authors match exactly then declare match
pos_trigger = mg.MatchTrigger()
pos_trigger.add_cond_rule('author_author_lev(ltuple, rtuple) > 0.7', 
                          feature_table=feature_table)
pos_trigger.add_cond_status(True)
pos_trigger.add_action(1)


True

time: 4.05 ms


In [51]:
res = mg.cv_matcher_and_trigger(dt, pos_trigger, development, 
                                exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                target_attr='gold', random_state = 10)

0%  100%
[#####] | ETA[sec]: 0.000 

time: 2.89 s



Total time elapsed: 2.878 sec


In [52]:
res['cv_stats']

,Metric,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,precision,5,0.750000,1.000000,0.800000,0.666667,0.750000,0.793333
1,recall,5,1.000000,0.800000,1.000000,1.000000,1.000000,0.960000
2,f1,5,0.857143,0.888889,0.888889,0.800000,0.857143,0.858413


time: 33.4 ms


In [53]:
#Apply DT matcher + positive trigger  over "test" set
dt.fit(table=development, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')
L_ = dt.predict(table=test, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
              target_attr='predicted',
              append=True, inplace=False)
M = pos_trigger.execute(L_, 'predicted', inplace=False)


time: 636 ms


In [54]:
# evaluate ML matcher + trigger
eval_summary = mg.eval_matches(M, 'gold', 'predicted')
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.9444444444444444, 1.0, 0.9714285714285714, 0.0, 1.0)

time: 10.5 ms


In [55]:
res = mg.cv_matcher_and_trigger(dt, pos_trigger, G, 
                                exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
                                target_attr='gold', random_state=0, k = 10)


0%      100%
[##########] | ETA[sec]: 0.000 

time: 7.38 s



Total time elapsed: 7.371 sec


In [56]:
res['cv_stats']

,Metric,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Fold 6,Fold 7,Fold 8,Fold 9,Fold 10,Mean score
0,precision,10,1.000000,1,0.800000,1,0.800000,1,0.750000,0.750000,0.666667,0.800000,0.856667
1,recall,10,0.666667,1,1.000000,1,1.000000,1,1.000000,1.000000,1.000000,1.000000,0.966667
2,f1,10,0.800000,1,0.888889,1,0.888889,1,0.857143,0.857143,0.800000,0.888889,0.898095


time: 36.9 ms


In [57]:
#--------Rule-based matcher + triggers--------#
# 1. write rules
# 2. add triggers

time: 676 µs


In [58]:
# Create rule-based matcher and add rules.
rm = mg.BooleanRuleMatcher()
rm.add_rule(['title_title_jac_qgm_3_qgm_3(ltuple, rtuple) > 0.6' 
             ], 
            feature_table=feature_table)


True

time: 2.68 ms


In [59]:
rm.add_rule(['author_author_jac_qgm_3_qgm_3(ltuple, rtuple) > 0.6'], 
            feature_table=feature_table)
rm.add_rule(['binding_binding_jac_qgm_3_qgm_3(ltuple, rtuple) > 0.5'], 
            feature_table=feature_table)

True

time: 2.84 ms


In [60]:
L.head()

,_id,ltable.id,rtable.id,ltable.title,ltable.author,rtable.title,rtable.author,gold
0,0,4086892,9780226156439,The Bedtrick: Tales of Sex and Masquerade,"Doniger, Wendy",Bedtrick : Tales of Sex and Masquerade,"Doniger, Wendy",1
1,1,535599,9780060655808,HarperCollins Study Bible-NRSV,"Meeks, Wayne A.",Harpercollins Study Bible Standard Version wit...,"Meeks, Wayne A.",1
2,3,1386538,9780130901262,Adaptive Filter Theory,"Haykin, Simon S.",Adaptive Filter Theory,"Haykin, Simon",1
3,5,15280600,9780217455008,"Catalogue of Earthquakes on the Pacific Coast,...","McAdie, Alexander","Catalogue of Earthquakes on the Pacific Coast,...","Mcadie, Alexander",1
4,6,15286798,9780217825382,"Cured by an Incurable, by Crowquill and Pencil...","Pseud, Crowquill","Cured by an Incurable, by Crowquill and Pencil...","Pseud, Crowquill",1


time: 12.2 ms


In [61]:
# Note here the input is candidate set !!!
X = rm.predict(table=L, target_attr='predicted', 
               append=True, inplace=False)

time: 667 ms


In [62]:
X.head(10)

,_id,ltable.id,rtable.id,ltable.title,ltable.author,rtable.title,rtable.author,gold,predicted
0,0,4086892,9780226156439,The Bedtrick: Tales of Sex and Masquerade,"Doniger, Wendy",Bedtrick : Tales of Sex and Masquerade,"Doniger, Wendy",1,1
1,1,535599,9780060655808,HarperCollins Study Bible-NRSV,"Meeks, Wayne A.",Harpercollins Study Bible Standard Version wit...,"Meeks, Wayne A.",1,1
2,3,1386538,9780130901262,Adaptive Filter Theory,"Haykin, Simon S.",Adaptive Filter Theory,"Haykin, Simon",1,1
3,5,15280600,9780217455008,"Catalogue of Earthquakes on the Pacific Coast,...","McAdie, Alexander","Catalogue of Earthquakes on the Pacific Coast,...","Mcadie, Alexander",1,1
4,6,15286798,9780217825382,"Cured by an Incurable, by Crowquill and Pencil...","Pseud, Crowquill","Cured by an Incurable, by Crowquill and Pencil...","Pseud, Crowquill",1,1
5,7,15295588,9780217940207,The Daring Adventures of Kit Carson and Fremon...,"Frmont, John Charles",Daring Adventures of Kit Carson and Fremont,"Frmont, John Charles",0,1
6,8,15334872,9780217270052,"Personal Efficiency, Applied Salesmanship, and...","Allen, Irving Ross","Personal Efficiency, Applied Salesmanship, and...","Allen, Irving Ross",1,1
7,9,15338150,9780217304399,The Universal Merchant; In Theory and Practice...,"Alldridge, W. J.",Universal Merchant,"Alldridge, W. J.",1,1
8,10,1566948,9780070479081,The Elusive Eden: A New History of California,"Rice, Richard B.",Elusive Eden : A New History of California,"Rice, Richard Bullough, Bonnie Bullough, Willi...",1,1
9,12,4564015,9780080446837,Vintage Papers from the Lancet,"Richardson, Ruth",Vintage Papers from the Lancet,NaN,0,1


time: 26.4 ms


In [63]:
X.head(10)

,_id,ltable.id,rtable.id,ltable.title,ltable.author,rtable.title,rtable.author,gold,predicted
0,0,4086892,9780226156439,The Bedtrick: Tales of Sex and Masquerade,"Doniger, Wendy",Bedtrick : Tales of Sex and Masquerade,"Doniger, Wendy",1,1
1,1,535599,9780060655808,HarperCollins Study Bible-NRSV,"Meeks, Wayne A.",Harpercollins Study Bible Standard Version wit...,"Meeks, Wayne A.",1,1
2,3,1386538,9780130901262,Adaptive Filter Theory,"Haykin, Simon S.",Adaptive Filter Theory,"Haykin, Simon",1,1
3,5,15280600,9780217455008,"Catalogue of Earthquakes on the Pacific Coast,...","McAdie, Alexander","Catalogue of Earthquakes on the Pacific Coast,...","Mcadie, Alexander",1,1
4,6,15286798,9780217825382,"Cured by an Incurable, by Crowquill and Pencil...","Pseud, Crowquill","Cured by an Incurable, by Crowquill and Pencil...","Pseud, Crowquill",1,1
5,7,15295588,9780217940207,The Daring Adventures of Kit Carson and Fremon...,"Frmont, John Charles",Daring Adventures of Kit Carson and Fremont,"Frmont, John Charles",0,1
6,8,15334872,9780217270052,"Personal Efficiency, Applied Salesmanship, and...","Allen, Irving Ross","Personal Efficiency, Applied Salesmanship, and...","Allen, Irving Ross",1,1
7,9,15338150,9780217304399,The Universal Merchant; In Theory and Practice...,"Alldridge, W. J.",Universal Merchant,"Alldridge, W. J.",1,1
8,10,1566948,9780070479081,The Elusive Eden: A New History of California,"Rice, Richard B.",Elusive Eden : A New History of California,"Rice, Richard Bullough, Bonnie Bullough, Willi...",1,1
9,12,4564015,9780080446837,Vintage Papers from the Lancet,"Richardson, Ruth",Vintage Papers from the Lancet,NaN,0,1


time: 29.5 ms


In [64]:
# Evaluate rule-based matcher performance
eval_summary = mg.eval_matches(X, 'gold', 'predicted')
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.7391304347826086, 0.9444444444444444, 0.8292682926829269, 2.0, 12.0)

time: 27.9 ms


In [65]:
 #Evaluate rule-based matcher performance
eval_summary = mg.eval_matches(X, 'gold', 'predicted')
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.7391304347826086, 0.9444444444444444, 0.8292682926829269, 2.0, 12.0)

time: 16.2 ms


In [66]:
mg.vis_debug_rm(rm, L, 'gold', feature_table)

time: 4.92 s


In [67]:
# address false positives
# Add triggers to address false negatives
# if authors match exactly then declare match
neg_trigger = mg.MatchTrigger()
neg_trigger.add_cond_rule(['lang_lang_lev(ltuple, rtuple) > 0.5'],
    feature_table=feature_table)
neg_trigger.add_cond_status(False)
neg_trigger.add_action(0)


True

time: 8.1 ms


In [68]:
Y = neg_trigger.execute(X, 'predicted', inplace=False)

time: 637 ms


In [69]:
eval_summary = mg.eval_matches(Y, 'gold', 'predicted')
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.7727272727272727, 0.9444444444444444, 0.85, 2.0, 10.0)

time: 29.6 ms


In [70]:
# choose ml matcher + trigger

time: 616 µs


In [71]:
# Extract feature vectors using subset of features
N = mg.extract_feat_vecs(C, feature_table=f)

time: 438 ms


In [72]:
len(N)

68

time: 1.71 ms


In [73]:
dt.fit(table=G, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')

time: 5.96 ms


In [74]:
# Impute values
N.fillna(0, inplace=True)

time: 2.12 ms


In [75]:
O = dt.predict(table=N, exclude_attrs=['_id', 'ltable.id', 'rtable.id'],
              target_attr='predicted', append=True, inplace=False)

time: 11.5 ms


In [76]:
O.head()

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,predicted
0,0,4086892,9780226156439,0.785714,1.000000,0,0.518519,1,1
1,1,535599,9780060655808,0.222222,1.000000,0,0.320000,1,1
2,2,12537042,9780030402432,0.243243,0.285714,0,0.025641,0,1
3,3,1386538,9780130901262,1.000000,0.785714,1,0.733333,1,1
4,4,13945932,9780205700424,0.375000,0.375000,0,0.733333,1,1


time: 18.1 ms


In [77]:
P = pos_trigger.execute(O, 'predicted', inplace=False)

time: 636 ms
